In [ ]:
import sqlite3
import pandas as pd
from pathlib import Path
import os
db_path="/example_data/example.db"
output_dir = "/output"
os.makedirs(Path(output_dir),exist_ok=True)
conn = sqlite3.connect(db_path)
pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)

,name
0,TripUpdates_poi
1,TripUpdates_poi_joint
2,stop_pairs


In [2]:
from src.poi_joint import prepare_stop_pairs
ob_window=10*60
gap=1
prepare_stop_pairs(db_path, gap, ob_window)

created index for (trip_id, vehicle_id, start_date, stop_sequence)


In [3]:
df_pairs = pd.read_sql_query("SELECT * FROM stop_pairs limit 5", conn)
for column in df_pairs.columns:
    print(column)

trip_id
start_date
vehicle_id
stop_i
seq_i
arr_delay_i
dep_delay_i
arr_time_i
duration_i
his_dwell_count_i
his_avg_delay_i
his_avg_dwell_i
stop_j
seq_j
arr_delay_j
dep_delay_j
arr_time_j
duration_j
travel_time
delay_change
hour_of_day
day_of_week


In [4]:
import sqlite3
conn = sqlite3.connect(db_path)

# Basic statistics
stats_query = """
SELECT 
    COUNT(*) as total_records,
    COUNT(DISTINCT trip_id) as n_trips,
    COUNT(DISTINCT stop_id) as n_stops,
    COUNT(DISTINCT start_date) as n_days,
    MIN(start_date) as first_date,
    MAX(start_date) as last_date
FROM TripUpdates_poi
"""
stats = pd.read_sql(stats_query, conn)
print(stats.to_string(index=False))


 total_records  n_trips  n_stops  n_days first_date last_date
         76595     2447      804       4   20250505  20250508


In [5]:
# Delay statistics - fixed for SQLite
delay_stats = """
SELECT 
    AVG(arr_delay) as mean_delay,
    MIN(arr_delay) as min_delay,
    MAX(arr_delay) as max_delay,
    COUNT(CASE WHEN arr_delay > 60 THEN 1 END) as late_count,
    COUNT(CASE WHEN arr_delay < -60 THEN 1 END) as early_count,
    COUNT(CASE WHEN arr_delay BETWEEN -60 AND 60 THEN 1 END) as ontime_count,
    COUNT(*) as total_count,
    ROUND(100.0 * COUNT(CASE WHEN arr_delay > 60 THEN 1 END) / COUNT(*), 2) as late_pct,
    ROUND(100.0 * COUNT(CASE WHEN arr_delay < -60 THEN 1 END) / COUNT(*), 2) as early_pct,
    ROUND(100.0 * COUNT(CASE WHEN arr_delay BETWEEN -60 AND 60 THEN 1 END) / COUNT(*), 2) as ontime_pct
FROM TripUpdates_poi
WHERE arr_delay IS NOT NULL
"""

print("\nDelay Statistics:")
print("(Negative = Early, Positive = Late, unit: seconds)")
delay_df = pd.read_sql(delay_stats, conn)
print(delay_df.to_string(index=False))

# Compute std using pandas
all_delays = pd.read_sql("SELECT arr_delay FROM TripUpdates_poi WHERE arr_delay IS NOT NULL", conn)
print(f"\nStandard Deviation of Delays: {all_delays['arr_delay'].std():.2f} seconds")


Delay Statistics:
(Negative = Early, Positive = Late, unit: seconds)
 mean_delay  min_delay  max_delay  late_count  early_count  ontime_count  total_count  late_pct  early_pct  ontime_pct
 112.457092      -2709       7617       43335         7670         25590        76595     56.58      10.01       33.41

Standard Deviation of Delays: 207.14 seconds


In [ ]:
gap = 1
ob_window = 20*60
from src.statistical_learning import compare_methods
compare_methods(db_path, gap, output_dir)

created index for (trip_id, vehicle_id, start_date, stop_sequence)
Loaded 67,634 observations
Remained 67,634 observations after outlier filtering
Treatment: delay_i (continuous, in seconds)
  Mean: 117.77 seconds
  Std: 199.02 seconds
  Range: [-2709.00, 7222.00]

Outcome: delay_j (continuous, in seconds)
  Mean: 119.88 seconds
  Std: 198.28 seconds

IDENTIFYING CONFOUNDERS

Confounders included:
  - hour_of_day
  - his_dwell_count_i
  - his_avg_delay_i
  - his_avg_dwell_i
  - is_morning_peak
  - is_evening_peak

Clean dataset: 67,634 observations

METHOD 1: NAIVE COMPARISON (BASELINE)
Naive estimate: β = 0.9062
Interpretation: Each 1-second increase in delay_i is associated
                with 0.9062 seconds increase in delay_j

METHOD 2: REGRESSION ADJUSTMENT
Regression-adjusted estimate: β = 0.9142
Interpretation: Controlling for confounders, each 1-second increase
                in delay_i causes 0.9142 seconds increase in delay_j

Regression coefficients:
  hour_of_day: 7.9097


In [7]:
gap = 2
from src.statistical_learning import compare_methods
compare_methods(db_path, gap, output_dir)

created index for (trip_id, vehicle_id, start_date, stop_sequence)
Loaded 59,883 observations
Remained 59,883 observations after outlier filtering
Treatment: delay_i (continuous, in seconds)
  Mean: 119.25 seconds
  Std: 198.51 seconds
  Range: [-2709.00, 7222.00]

Outcome: delay_j (continuous, in seconds)
  Mean: 122.46 seconds
  Std: 199.37 seconds

IDENTIFYING CONFOUNDERS

Confounders included:
  - hour_of_day
  - his_dwell_count_i
  - his_avg_delay_i
  - his_avg_dwell_i
  - is_morning_peak
  - is_evening_peak

Clean dataset: 59,883 observations

METHOD 1: NAIVE COMPARISON (BASELINE)
Naive estimate: β = 0.8817
Interpretation: Each 1-second increase in delay_i is associated
                with 0.8817 seconds increase in delay_j

METHOD 2: REGRESSION ADJUSTMENT
Regression-adjusted estimate: β = 0.8906
Interpretation: Controlling for confounders, each 1-second increase
                in delay_i causes 0.8906 seconds increase in delay_j

Regression coefficients:
  hour_of_day: 7.5668


In [8]:
gap = 3
from src.statistical_learning import compare_methods
compare_methods(db_path, gap, output_dir)

created index for (trip_id, vehicle_id, start_date, stop_sequence)
Loaded 52,778 observations
Remained 52,778 observations after outlier filtering
Treatment: delay_i (continuous, in seconds)
  Mean: 120.72 seconds
  Std: 198.14 seconds
  Range: [-2275.00, 7222.00]

Outcome: delay_j (continuous, in seconds)
  Mean: 123.40 seconds
  Std: 200.58 seconds

IDENTIFYING CONFOUNDERS

Confounders included:
  - hour_of_day
  - his_dwell_count_i
  - his_avg_delay_i
  - his_avg_dwell_i
  - is_morning_peak
  - is_evening_peak

Clean dataset: 52,778 observations

METHOD 1: NAIVE COMPARISON (BASELINE)
Naive estimate: β = 0.8533
Interpretation: Each 1-second increase in delay_i is associated
                with 0.8533 seconds increase in delay_j

METHOD 2: REGRESSION ADJUSTMENT
Regression-adjusted estimate: β = 0.8637
Interpretation: Controlling for confounders, each 1-second increase
                in delay_i causes 0.8637 seconds increase in delay_j

Regression coefficients:
  hour_of_day: 8.1825
